In [5]:
# from statsmodels.regression.rolling import RollingOLS
# import pandas_datareader.data as web
# import matplotlib.pyplot as plt
# import statsmodels.api as sm
# import pandas as pd
# import numpy as np
# import datetime as dt
# import yfinance as yf
# import pandas_ta
# import warnings
# warnings.filterwarnings('ignore')

# sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]

# sp500['Symbol'] = sp500['Symbol'].str.replace('.', '-')

# symbols_list = sp500['Symbol'].unique().tolist()

# end_date = '2024-05-11'

# start_date = pd.to_datetime(end_date)-pd.DateOffset(365*8)

# df = yf.download(tickers=symbols_list,
#                  start=start_date,
#                  end=end_date).stack()

# df.index.names = ['date', 'ticker']

# df.columns = df.columns.str.lower()


In [6]:
# # Calculate features and technical indicators per stock
# # Garman-Klass Volatility
# # RSI
# # Bollinger Bands
# # ATR
# # MACD
# # Dollar Volume

# df['garman_klass_vol'] = ((np.log(df['high'])-np.log(df['low']))**2)/2-(2*np.log(2)-1)*((np.log(df['adj close'])-np.log(df['open']))**2)

# df['rsi'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.rsi(close=x, length=20))

# df['bb_low'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,0])
                                                          
# df['bb_mid'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,1])
                                                          
# df['bb_high'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,2])

# def compute_atr(stock_data):
#     atr = pandas_ta.atr(high=stock_data['high'],
#                         low=stock_data['low'],
#                         close=stock_data['close'],
#                         length=14)
#     return atr.sub(atr.mean()).div(atr.std())

# df['atr'] = df.groupby(level=1, group_keys=False).apply(compute_atr)

# def compute_macd(close):
#     macd = pandas_ta.macd(close=close, length=20).iloc[:,0]
#     return macd.sub(macd.mean()).div(macd.std())

# df['macd'] = df.groupby(level=1, group_keys=False)['adj close'].apply(compute_macd)

df['dollar_volume'] = (df['adj close']*df['volume'])/1e6

df

Price               adj close       close        high         low        open  \
date       ticker                                                               
2015-09-29 A        31.483557   33.740002   34.060001   33.240002   33.360001   
           AAL      37.361629   39.180000   39.770000   38.790001   39.049999   
           AAPL     24.651134   27.264999   28.377501   26.965000   28.207500   
           ABBV     36.334888   52.790001   54.189999   51.880001   53.099998   
           ABT      33.478706   39.500000   40.150002   39.029999   39.259998   
...                       ...         ...         ...         ...         ...   
2023-09-26 XYL      88.968475   89.519997   90.849998   89.500000   90.379997   
           YUM     122.811577  124.010002  124.739998  123.449997  124.239998   
           ZBH     111.782722  112.459999  117.110001  112.419998  116.769997   
           ZBRA    223.960007  223.960007  226.649994  222.580002  225.970001   
           ZTS     175.547714  176.869995  178.449997  176.270004  176.580002   

Price                   volume  garman_klass_vol        rsi    bb_low  \
date       ticker                                                       
2015-09-29 A         2252400.0         -0.000998        NaN       NaN   
           AAL       7478800.0         -0.000443        NaN       NaN   
           AAPL    293461600.0         -0.005712        NaN       NaN   
           ABBV     12842800.0         -0.054656        NaN       NaN   
           ABT      12287500.0         -0.009402        NaN       NaN   
...                        ...               ...        ...       ...   
2023-09-26 XYL       1322400.0          0.000016  26.146751  4.488345   
           YUM       1500600.0          0.000002  36.057182  4.816569   
           ZBH       3610500.0          0.000099  31.893231  4.748085   
           ZBRA       355400.0          0.000133  29.494977  5.400991   
           ZTS       1463200.0          0.000062  42.623488  5.156108   

Price                bb_mid   bb_high       atr      macd  dollar_volume  
date       ticker                                                         
2015-09-29 A            NaN       NaN       NaN       NaN      70.913563  
           AAL          NaN       NaN       NaN       NaN     279.420155  
           AAPL         NaN       NaN       NaN       NaN    7234.161370  
           ABBV         NaN       NaN       NaN       NaN     466.641705  
           ABT          NaN       NaN       NaN       NaN     411.369604  
...                     ...       ...       ...       ...            ...  
2023-09-26 XYL     4.570270  4.652196  0.033800 -2.159188     117.651912  
           YUM     4.846536  4.876502  0.142547 -1.363696     184.291052  
           ZBH     4.787752  4.827420 -0.381708 -0.881067     403.591519  
           ZBRA    5.539167  5.677342 -0.057389 -1.600791      79.595386  
           ZTS     5.214922  5.273735  0.651515 -1.188278     256.861415  

[988898 rows x 14 columns]

In [31]:
# Filter top 150 most liquid stocks for each month

# last_cols = [c for c in df.columns.unique(0) if c not in ['dollar_volume', 'volume', 'open',
#                                                           'high', 'low', 'close']]

# data = (pd.concat([df.unstack('ticker')['dollar_volume'].resample('M').mean().stack('ticker').to_frame('dollar_volume'),
#                    df.unstack()[last_cols].resample('M').last().stack('ticker')],
#                   axis=1)).dropna()


In [36]:
# Calulate the 5 year rolling average dollar volume for each stock 

# data['dollar_volume'].unstack('ticker').rolling(5*12).mean()

# data['dollar_vol_rank'] = (data.groupby('date')['dollar_volume'].rank(ascending=False))

# data[data['dollar_vol_rank'] < 150].drop(['dollar_volume', 'dollar_vol_rank'], axis=1)

adj close  garman_klass_vol        rsi    bb_low  \
date       ticker                                                      
2015-11-30 AAL      39.429935         -0.000966  40.719003  3.672028   
           AAPL     26.854137         -0.003307  55.537344  3.281679   
           ABBV     40.393524         -0.059549  49.376853  3.726695   
           ABT      38.293560         -0.011216  56.962491  3.656055   
           ACN      93.640091         -0.007419  59.371848  4.518436   
...                       ...               ...        ...       ...   
2023-09-30 XOM     114.292961          0.000026  59.440189  4.695106   
           MRNA     98.120003          0.000146  38.747314  4.582514   
           UBER     44.270000          0.000441  45.005268  3.806654   
           ABNB    132.279999          0.000213  44.494127  4.857047   
           KVUE     20.006193         -0.000159  35.706328  3.014275   

                     bb_mid   bb_high       atr      macd  
date       ticker                                          
2015-11-30 AAL     3.749832  3.827636  0.190822 -0.418772  
           AAPL    3.324991  3.368302 -0.967900 -0.142790  
           ABBV    3.774943  3.823191 -0.526809  0.145676  
           ABT     3.677909  3.699763 -1.064842  0.335556  
           ACN     4.542765  4.567094 -1.121687  0.039552  
...                     ...       ...       ...       ...  
2023-09-30 XOM     4.735205  4.775304  0.601335  1.400623  
           MRNA    4.685332  4.788149 -0.529511 -0.376899  
           UBER    3.862227  3.917801 -0.746098 -0.133973  
           ABNB    4.940924  5.024801 -1.006939 -0.037854  
           KVUE    3.089221  3.164168 -0.899746 -1.435621  

[14155 rows x 8 columns]

In [46]:
# Calculate monthly returns for different horizon as features

# Use lags for 1, 2, 3, 6, 9, 12 months gives 6 different lags

def calculate_returns(df):

    outlier_cutoff = 0.005

    lags = [1, 2, 3, 6, 9, 12]

    for lag in lags:
        
        df[f'return_{lag}m'] = (df['adj close']
                            .pct_change(lag)
                            .pipe(lambda x: x.clip(lower=x.quantile(outlier_cutoff),
                                                    upper=x.quantile(1-outlier_cutoff)))
                            .add(1)
                            .pow(1/lag)
                            .sub(1))
    return df
    
data = data.groupby(level=1, group_keys=False).apply(calculate_returns).dropna()

data

dollar_volume   adj close  garman_klass_vol        rsi  \
date       ticker                                                           
2016-11-30 A          107.765977   41.481586         -0.002041  44.135570   
           AAL        298.198623   44.876518          0.000189  71.397662   
           AAPL      3502.993187   25.646255         -0.002590  48.015275   
           ABBV       404.410005   43.867043         -0.044457  49.727659   
           ABT        294.927862   33.270931         -0.008821  39.665361   
...                          ...         ...               ...        ...   
2023-09-30 CTVA       267.143825   50.462921          0.000101  44.013665   
           CARR       252.559506   52.174889          0.000971  42.987007   
           OTIS       154.983159   78.671936         -0.000020  33.116253   
           ABNB      1633.500725  132.279999          0.000213  44.494127   
           CEG        196.971016  108.026917          0.000171  55.245482   

                     bb_low    bb_mid   bb_high       atr      macd  \
date       ticker                                                     
2016-11-30 A       3.729552  3.772138  3.814724 -0.888157 -0.331612   
           AAL     3.656155  3.770906  3.885658  0.263773  2.344163   
           AAPL    3.249456  3.278521  3.307586 -1.011998 -0.314781   
           ABBV    3.722149  3.796301  3.870452 -0.617478 -0.257987   
           ABT     3.530306  3.566758  3.603210 -0.981659 -0.575624   
...                     ...       ...       ...       ...       ...   
2023-09-30 CTVA    3.914700  3.948840  3.982979 -0.803714 -0.385747   
           CARR    3.955645  4.025057  4.094468  1.818338 -1.120464   
           OTIS    4.374104  4.419394  4.464683 -1.028320 -1.534536   
           ABNB    4.857047  4.940924  5.024801 -1.006939 -0.037854   
           CEG     4.653660  4.693834  4.734007 -0.436215  0.366876   

                   dollar_vol_rank  return_1m  return_2m  return_3m  \
date       ticker                                                     
2016-11-30 A                 290.0   0.009410  -0.033586  -0.020946   
           AAL               104.0   0.146730   0.127700   0.086493   
           AAPL                3.0  -0.021601  -0.008718   0.015430   
           ABBV               62.0   0.089996  -0.013645  -0.014453   
           ABT               106.0  -0.029817  -0.048194  -0.030325   
...                            ...        ...        ...        ...   
2023-09-30 CTVA              190.0   0.005543  -0.049805  -0.038402   
           CARR              204.0  -0.082332  -0.059093   0.019790   
           OTIS              320.0  -0.073174  -0.064483  -0.036530   
           ABNB               16.0   0.005549  -0.067704   0.010603   
           CEG               250.0   0.041571   0.060899   0.059165   

                   return_6m  return_9m  return_12m  
date       ticker                                    
2016-11-30 A       -0.006214   0.019227    0.005104  
           AAL      0.065515   0.014886    0.010841  
           AAPL     0.018834   0.016838   -0.003828  
           ABBV    -0.002749   0.015097    0.006898  
           ABT     -0.004603   0.000125   -0.011648  
...                      ...        ...         ...  
2023-09-30 CTVA    -0.027278  -0.015197   -0.008915  
           CARR     0.025342   0.028590    0.034814  
           OTIS    -0.009024   0.002663    0.019569  
           ABNB     0.010289   0.048608    0.019401  
           CEG      0.056503   0.026987    0.023339  

[40703 rows x 16 columns]

In [73]:
# Download Fama-French factors and calculate rolling factor betas

factor_data = web.DataReader('F-F_Research_Data_5_Factors_2x3',
                               'famafrench',
                               start='2010')[0].drop('RF', axis=1)

factor_data.index = factor_data.index.to_timestamp()

factor_data

,Mkt-RF,SMB,HML,RMW,CMA
Date,,,,,
2010-01-01,-3.36,0.34,0.43,-1.27,0.46
2010-02-01,3.40,1.51,3.23,-0.27,1.43
2010-03-01,6.31,1.85,2.21,-0.65,1.69
2010-04-01,2.00,4.98,2.89,0.69,1.72
2010-05-01,-7.89,0.04,-2.44,1.30,-0.21
...,...,...,...,...,...
2023-11-01,8.84,-0.12,1.64,-3.91,-1.00
2023-12-01,4.87,7.32,4.93,-3.07,1.32
2024-01-01,0.71,-5.74,-2.38,0.69,-0.96
